In [11]:
!pip install sidetable


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd
import sidetable as stb

In [13]:
DATA_PATH = "data/car_prices.csv"

In [14]:
df_data = pd.read_csv(DATA_PATH)

# Exploration

In [15]:
df_data.shape

(558837, 16)

In [16]:
df_data.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object

In [17]:
df_data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [18]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


In [19]:
df_data.describe()

,year,condition,odometer,mmr,sellingprice
count,558837.000000,547017.000000,558743.000000,558799.000000,558825.000000
mean,2010.038927,30.672365,68320.017767,13769.377495,13611.358810
std,3.966864,13.402832,53398.542821,9679.967174,9749.501628
min,1982.000000,1.000000,1.000000,25.000000,1.000000
25%,2007.000000,23.000000,28371.000000,7100.000000,6900.000000
50%,2012.000000,35.000000,52254.000000,12250.000000,12100.000000
75%,2013.000000,42.000000,99109.000000,18300.000000,18200.000000
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000


In [20]:
NUM_COLUMNS = ["year", "condition", "odometer", "mmr", "sellingprice"]
DATE_COLUMNS = ["saledate"]
TEXT_COLUMNS = [c for c in df_data.columns if c not in NUM_COLUMNS + DATE_COLUMNS + ["vin"]]

# Nettoyage

## Suppression des colonnes inutiles

In [21]:
df_data = df_data.drop(columns=["vin"])

## Valeurs manquantes

In [22]:
df_data.stb.missing()

,missing,total,percent
transmission,65352,558837,11.694287
body,13195,558837,2.361154
condition,11820,558837,2.115107
trim,10651,558837,1.905922
model,10399,558837,1.860829
make,10301,558837,1.843292
color,749,558837,0.134028
interior,749,558837,0.134028
odometer,94,558837,0.016821
mmr,38,558837,0.006800


In [23]:
df_data.columns.drop("transmission")

Index(['year', 'make', 'model', 'trim', 'body', 'state', 'condition',
       'odometer', 'color', 'interior', 'seller', 'mmr', 'sellingprice',
       'saledate'],
      dtype='object')

In [24]:
row_with_nan = df_data.shape[0] - df_data[df_data.columns.drop("transmission")].dropna().shape[0]

In [25]:
print(f"Nombre de lignes avec au moins une valeur nan (sans la colonne transmission) : {row_with_nan}") 
print(f"Soit {(row_with_nan / df_data.shape[0]) * 100}% des lignes du jeu de données.")

Nombre de lignes avec au moins une valeur nan (sans la colonne transmission) : 25189
Soit 4.507396611176425% des lignes du jeu de données.


In [26]:
row_with_nan = df_data.shape[0] - df_data[["sellingprice", "saledate", "make", "model"]].dropna().shape[0]

In [27]:
print(f"Nombre de lignes avec au moins une valeur nan (sans la colonne transmission) : {row_with_nan}") 
print(f"Soit {(row_with_nan / df_data.shape[0]) * 100}% des lignes du jeu de données.")

Nombre de lignes avec au moins une valeur nan (sans la colonne transmission) : 10411
Soit 1.8629761451013445% des lignes du jeu de données.


In [28]:
#df_data = df_data[df_data.columns.drop(["transmission", "body", "condition", "trim"])].dropna()

In [29]:
df_data = df_data.dropna(how="any", subset=["sellingprice", "saledate", "make", "model"])

In [30]:
df_data.shape

(548426, 15)

In [52]:
df_data.stb.missing()

,missing,total,percent
condition,11733,548426,2.139395
odometer,90,548426,0.016411
mmr,26,548426,0.004741
trim,0,548426,0.000000
year,0,548426,0.000000
make,0,548426,0.000000
model,0,548426,0.000000
state,0,548426,0.000000
transmission,0,548426,0.000000
body,0,548426,0.000000


In [32]:
df_data.loc[:, TEXT_COLUMNS] = df_data[TEXT_COLUMNS].fillna("missing")

## Nettoyage des colonnes de texte

### Le fabriquant

In [33]:
df_data["make"].nunique()

96

In [34]:
df_data["make"].unique()

array(['Kia', 'BMW', 'Volvo', 'Nissan', 'Chevrolet', 'Audi', 'Ford',
       'Hyundai', 'Buick', 'Cadillac', 'Acura', 'Lexus', 'Infiniti',
       'Jeep', 'Mercedes-Benz', 'Mitsubishi', 'Mazda', 'MINI',
       'Land Rover', 'Lincoln', 'lincoln', 'Jaguar', 'Volkswagen',
       'Toyota', 'Subaru', 'Scion', 'Porsche', 'bmw', 'Dodge', 'FIAT',
       'Chrysler', 'ford', 'Ferrari', 'Honda', 'GMC', 'mitsubishi', 'Ram',
       'smart', 'chevrolet', 'Bentley', 'chrysler', 'pontiac', 'Pontiac',
       'Saturn', 'Maserati', 'Mercury', 'HUMMER', 'landrover', 'cadillac',
       'land rover', 'mercedes', 'mazda', 'toyota', 'lexus', 'gmc truck',
       'honda', 'nissan', 'porsche', 'Saab', 'Suzuki', 'dodge', 'subaru',
       'Oldsmobile', 'oldsmobile', 'hyundai', 'jeep', 'Isuzu', 'dodge tk',
       'Geo', 'acura', 'volkswagen', 'suzuki', 'kia', 'audi',
       'Rolls-Royce', 'gmc', 'maserati', 'mazda tk', 'mercury', 'buick',
       'hyundai tk', 'mercedes-b', 'vw', 'Daewoo', 'chev truck',
       'ford t

In [35]:
df_data.loc[:, "make"] = df_data["make"].apply(lambda x: x.lower())

In [36]:
df_data["make"].unique()

array(['kia', 'bmw', 'volvo', 'nissan', 'chevrolet', 'audi', 'ford',
       'hyundai', 'buick', 'cadillac', 'acura', 'lexus', 'infiniti',
       'jeep', 'mercedes-benz', 'mitsubishi', 'mazda', 'mini',
       'land rover', 'lincoln', 'jaguar', 'volkswagen', 'toyota',
       'subaru', 'scion', 'porsche', 'dodge', 'fiat', 'chrysler',
       'ferrari', 'honda', 'gmc', 'ram', 'smart', 'bentley', 'pontiac',
       'saturn', 'maserati', 'mercury', 'hummer', 'landrover', 'mercedes',
       'gmc truck', 'saab', 'suzuki', 'oldsmobile', 'isuzu', 'dodge tk',
       'geo', 'rolls-royce', 'mazda tk', 'hyundai tk', 'mercedes-b', 'vw',
       'daewoo', 'chev truck', 'ford tk', 'plymouth', 'ford truck',
       'tesla', 'airstream', 'dot', 'aston martin', 'fisker',
       'lamborghini', 'lotus'], dtype=object)

In [37]:
df_data.loc[:, "make"] = df_data["make"].str.replace("vw", "volkswagen")
df_data.loc[:, "make"] = df_data["make"].str.replace("mercedes-b", "mercedes")
df_data.loc[:, "make"] = df_data["make"].str.replace(" tk", "")
df_data.loc[:, "make"] = df_data["make"].str.replace(" truck", "")
#df_data.loc[:, "make"] = df_data["make"].str.replace("chev", "chevrolet")
#df_data.loc[:, "make"] = df_data["make"].str.replace("rolet", "chevrolet")

In [38]:
df_data["make"].unique()

array(['kia', 'bmw', 'volvo', 'nissan', 'chevrolet', 'audi', 'ford',
       'hyundai', 'buick', 'cadillac', 'acura', 'lexus', 'infiniti',
       'jeep', 'mercedesenz', 'mitsubishi', 'mazda', 'mini', 'land rover',
       'lincoln', 'jaguar', 'volkswagen', 'toyota', 'subaru', 'scion',
       'porsche', 'dodge', 'fiat', 'chrysler', 'ferrari', 'honda', 'gmc',
       'ram', 'smart', 'bentley', 'pontiac', 'saturn', 'maserati',
       'mercury', 'hummer', 'landrover', 'mercedes', 'saab', 'suzuki',
       'oldsmobile', 'isuzu', 'geo', 'rolls-royce', 'daewoo', 'chev',
       'plymouth', 'tesla', 'airstream', 'dot', 'aston martin', 'fisker',
       'lamborghini', 'lotus'], dtype=object)

## Nettoyage de la colonne date

In [46]:
!pip install dateparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 44.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [48]:
import dateparser

In [49]:
dateparser.parse('Tue Dec 16 2014 12:30:00 GMT-0800 (PST)')

datetime.datetime(2014, 12, 16, 12, 30, tzinfo=<StaticTzInfo 'UTC\-08:00'>)

In [83]:
df_data[DATE_COLUMNS[0]] = df_data[DATE_COLUMNS[0]].apply(dateparser.parse)

In [85]:
#df_data.loc[:, DATE_COLUMNS[0]] = dd

In [86]:
df_data.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object

In [88]:
df_data.head()

,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,kia,Sorento,LX,SUV,automatic,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,2014-12-16 12:30:00-08:00
1,2015,kia,Sorento,LX,SUV,automatic,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,2014-12-16 12:30:00-08:00
2,2014,bmw,3 Series,328i SULEV,Sedan,automatic,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,2015-01-15 04:30:00-08:00
3,2015,volvo,S60,T5,Sedan,automatic,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,2015-01-29 04:30:00-08:00
4,2014,bmw,6 Series Gran Coupe,650i,Sedan,automatic,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,2014-12-18 12:30:00-08:00


In [94]:
df_data.loc[:, DATE_COLUMNS[0]] = pd.to_datetime(df_data["saledate"], utc=True, errors="coerce")

In [95]:
df_data.dtypes

year                          int64
make                         object
model                        object
trim                         object
body                         object
transmission                 object
state                        object
condition                   float64
odometer                    float64
color                        object
interior                     object
seller                       object
mmr                         float64
sellingprice                float64
saledate        datetime64[ns, UTC]
dtype: object

In [96]:
df_data.stb.missing()

,missing,total,percent
condition,11733,548426,2.139395
odometer,90,548426,0.016411
saledate,26,548426,0.004741
mmr,26,548426,0.004741
year,0,548426,0.000000
make,0,548426,0.000000
model,0,548426,0.000000
state,0,548426,0.000000
transmission,0,548426,0.000000
body,0,548426,0.000000


In [99]:
df_data = df_data.dropna(how="any", subset=["saledate"])

In [100]:
df_data.stb.missing()

,missing,total,percent
condition,11707,548400,2.134756
odometer,90,548400,0.016411
year,0,548400,0.000000
trim,0,548400,0.000000
body,0,548400,0.000000
make,0,548400,0.000000
model,0,548400,0.000000
state,0,548400,0.000000
transmission,0,548400,0.000000
color,0,548400,0.000000


# Enregistrement des données au format CSV

In [101]:
df_data.to_csv("data/car_prices_clean.csv", index=False)